In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
)
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize global variables
result_table = None
result_table_widget = None
actual_values = None
predicted_values = None


# Function to reset the result table
def reset_table():
    global result_table, result_table_widget
    result_table = pd.DataFrame(columns=["Metric", "Value"])
    result_table_widget = widgets.Output()


# Functions to calculate metrics and update the table
def update_table(metric, value):
    global result_table, result_table_widget
    result_table = result_table.append({"Metric": metric, "Value": value}, ignore_index=True)
    with result_table_widget:
        clear_output(wait=True)
        display(result_table)


def calculate_regression_metric(metric):
    global actual_values, predicted_values
    if metric == "MAE":
        update_table("Mean Absolute Error", mean_absolute_error(actual_values, predicted_values))
    elif metric == "MSE":
        update_table("Mean Squared Error", mean_squared_error(actual_values, predicted_values))
    elif metric == "RMSE":
        update_table("Root Mean Squared Error", np.sqrt(mean_squared_error(actual_values, predicted_values)))
    elif metric == "R2":
        update_table("R-Squared", r2_score(actual_values, predicted_values))
    elif metric == "Adj R2":
        n = len(actual_values)
        p = predicted_values.shape[1] if len(predicted_values.shape) > 1 else 1
        r2 = r2_score(actual_values, predicted_values)
        adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
        update_table("Adjusted R-Squared", adj_r2)


def calculate_classification_metric(metric):
    global actual_values, predicted_values
    if metric == "Accuracy":
        update_table("Accuracy", accuracy_score(actual_values, predicted_values))
    elif metric == "AUC":
        update_table("AUC-ROC", roc_auc_score(actual_values, predicted_values))
    elif metric == "F1":
        update_table("F1 Score", f1_score(actual_values, predicted_values))
    elif metric == "Precision":
        update_table("Precision", precision_score(actual_values, predicted_values))
    elif metric == "Recall":
        update_table("Recall", recall_score(actual_values, predicted_values))


# Dropdowns for selecting actual and predicted values
def setup_dropdowns(df):
    actual_dropdown = widgets.Dropdown(
        options=df.columns,
        description="Actual:",
        layout=widgets.Layout(width="50%"),
    )
    predicted_dropdown = widgets.Dropdown(
        options=df.columns,
        description="Predicted:",
        layout=widgets.Layout(width="50%"),
    )

    def handle_selection(_):
        global actual_values, predicted_values
        try:
            actual_values = df[actual_dropdown.value].values
            predicted_values = df[predicted_dropdown.value].values
            display(widgets.HTML("<b>Values successfully loaded. You can now calculate metrics.</b>"))
        except Exception as e:
            display(widgets.HTML(f"<b>Error: {str(e)}</b>"))

    load_button = widgets.Button(description="Load Values", button_style="success")
    load_button.on_click(handle_selection)

    return widgets.VBox([actual_dropdown, predicted_dropdown, load_button])


# Metric Button Generators
def generate_metric_buttons(metrics, task_type):
    buttons = []
    for metric, tooltip in metrics.items():
        button = widgets.Button(description=metric, tooltip=tooltip, button_style="info")
        button.on_click(lambda _, metric=metric: metric_click_handler(metric, task_type))
        buttons.append(button)
    return widgets.VBox(buttons)


# Metric Click Handler
def metric_click_handler(metric, task_type):
    global actual_values, predicted_values
    if actual_values is None or predicted_values is None:
        display(widgets.HTML("<b>Please select actual and predicted values first!</b>"))
        return
    if task_type == "regression":
        if len(set(actual_values)) > 2:  # Assuming regression has continuous targets
            calculate_regression_metric(metric)
        else:
            display(widgets.HTML("<b>These metrics are not applicable to classification problems.</b>"))
    elif task_type == "classification":
        if len(set(actual_values)) <= 2:  # Assuming classification has binary or discrete targets
            calculate_classification_metric(metric)
        else:
            display(widgets.HTML("<b>These metrics are not applicable to regression problems.</b>"))


# Metric dictionaries
regression_metrics = {
    "MAE": "Mean Absolute Error",
    "MSE": "Mean Squared Error",
    "RMSE": "Root Mean Squared Error",
    "R2": "R-Squared",
    "Adj R2": "Adjusted R-Squared",
}

classification_metrics = {
    "Accuracy": "Accuracy Score",
    "AUC": "Area Under the Curve",
    "F1": "F1 Score",
    "Precision": "Precision Score",
    "Recall": "Recall Score",
}


# Start tool
def start_tool(df):
    def show_regression():
        reset_table()  # Clear previous results
        input_box = setup_dropdowns(df)
        display(input_box, generate_metric_buttons(regression_metrics, "regression"), result_table_widget)

    def show_classification():
        reset_table()  # Clear previous results
        input_box = setup_dropdowns(df)
        display(input_box, generate_metric_buttons(classification_metrics, "classification"), result_table_widget)

    reset_table()  # Initialize table
    regression_button = widgets.Button(
        description="Regression Evaluation Metrics", button_style="primary", tooltip="Click to show regression metrics"
    )
    classification_button = widgets.Button(
        description="Classification Evaluation Metrics",
        button_style="primary",
        tooltip="Click to show classification metrics",
    )

    regression_button.on_click(lambda _: show_regression())
    classification_button.on_click(lambda _: show_classification())

    display(widgets.HBox([regression_button, classification_button]))


In [5]:
# Example Usage
df = pd.DataFrame({
    'r_actual': [3, 5, 2, 7, 8],
    'r_predicted': [2.5, 5.1, 2.0, 7.2, 7.8],
    'c_actual': [1, 0, 0, 1, 1],
    'c_predicted': [0, 0, 1, 1, 1]
})
start_tool(df)


Output()

HTML(value='<b>Values successfully loaded. You can now calculate metrics.</b>')

Output()

HTML(value='<b>Values successfully loaded. You can now calculate metrics.</b>')